In [1]:
from core.data import from_dir, random_resample

data_path = 'data/chest_xray/train'

2025-02-24 18:03:32.707103: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-24 18:03:32.707402: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-24 18:03:32.709009: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-24 18:03:32.713067: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740416612.720165   48585 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740416612.72

In [2]:
resize = (96, 96)

In [3]:
training_selector = lambda s: s.startswith('chest_xray/train/') and s.endswith('.jpeg')
(x, y) = from_dir(data_path, resize=resize, resample=random_resample)

In [4]:
x_batch, y_batch = x[:10], y[:10]

In [5]:
from core.layers import RandomCutout

cut = RandomCutout(32, 2)(x_batch)

2025-02-24 18:03:35.803807: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [6]:
NEG_IDX = 0
POS_IDX = 1
TOTAL   = len(y)
N       = len(list(filter(lambda j: j == 0, y)))
P       = len(list(filter(lambda j: j == 1, y)))
print(f'#Positives = {P} ({P/TOTAL*100:.2f}%)')
print(f'#Negatives = {N} ({N/TOTAL*100:.2f}%)')

#Positives = 3883 (74.22%)
#Negatives = 1349 (25.78%)


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(x, y, test_size=0.2, random_state=42)
total_train = len(Y_train)
N_train     = len(list(filter(lambda j: j == 0, Y_train)))
P_train     = len(list(filter(lambda j: j == 1, Y_train)))
print(f'#Positives (train) = {P_train} ({P_train/total_train*100:.2f}%)')
print(f'#Negatives (train) = {N_train} ({N_train/total_train*100:.2f}%)')
total_val   = len(Y_val)
N_val       = len(list(filter(lambda j: j == 0, Y_val)))
P_val       = len(list(filter(lambda j: j == 1, Y_val)))
print(f'#Positives (val)   = {P_val} ({P_val/total_val*100:.2f}%)')
print(f'#Negatives (val)   = {N_val} ({N_val/total_val*100:.2f}%)')

#Positives (train) = 3095 (73.95%)
#Negatives (train) = 1090 (26.05%)
#Positives (val)   = 788 (75.26%)
#Negatives (val)   = 259 (24.74%)


In [8]:
import numpy as np
from core.imb import class_weights

print(class_weights(Y_train))

{0: 0.36024048559559513, 1: 0.1397595144044049}


In [9]:
import numpy as np

def apply_cutmix(x, y):
    shf = list(range(len(x)))
    np.random.shuffle(shf)
    x_mix = []
    y_mix = []
    for i in range(len(x)):
        a = (x[i], y[i])
        b = (x[shf[i]], y[shf[i]])
        xc, yc = preprocessing.cutmix(a, b)
        x_mix.append(xc)
        y_mix.append(yc)
    return np.asarray(x_mix), np.asarray(y_mix)

In [10]:
from keras import callbacks as cb, metrics as metr, optimizers
from core import create_model, preprocessing, imb

callbacks = [cb.EarlyStopping(patience=5, verbose=1)]
model = create_model('resnet-18', resize)
metrics = [metr.BinaryCrossentropy(name='loss'), metr.BinaryCrossentropy('val_loss'), metr.Recall(name='recall'), metr.Recall(name='val_recall')]
optimizer = optimizers.Adam()
loss = lambda t, y: losses.weighted_bce(t, y, w)

In [11]:
from core import loop


history = loop.training_loop(
    X_train, Y_train, X_val, Y_val,
    model,
    4,
    64,
    metrics,
    optimizer,
    callbacks,
    loss,
    apply_cutmix)
history

Epoch 1/4
 1/64 ━━━━━━━━━━━━━━━━━━━━ 2:19 2s/stepTraining loss (for one batch) at step 0: 0.0003
Seen so far: 64 samples
66/64 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/steploss: 0.4274083077907562,recall: 0.8257964849472046


2025-02-24 03:42:23.499914: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


val_loss: 0.6140022277832031,val_recall: 1.0
64/64 ━━━━━━━━━━━━━━━━━━━━ 15s 210ms/step - loss: 0.4274 - val_loss: 2.9629e-04
Epoch 2/4
 1/64 ━━━━━━━━━━━━━━━━━━━━ 11s 186ms/stepTraining loss (for one batch) at step 0: 0.0003
Seen so far: 64 samples
66/64 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/steploss: 0.3620903491973877,recall: 0.8279486894607544


2025-02-24 03:42:36.253681: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


val_loss: 1.006507158279419,val_recall: 1.0
64/64 ━━━━━━━━━━━━━━━━━━━━ 13s 197ms/step - loss: 0.3621 - val_loss: 4.8570e-04
Epoch 3/4
 1/64 ━━━━━━━━━━━━━━━━━━━━ 11s 180ms/stepTraining loss (for one batch) at step 0: 0.0001
Seen so far: 64 samples
66/64 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/steploss: 0.34301382303237915,recall: 0.8256739377975464
val_loss: 0.9321171045303345,val_recall: 1.0
64/64 ━━━━━━━━━━━━━━━━━━━━ 12s 196ms/step - loss: 0.3430 - val_loss: 4.4981e-04
Epoch 4/4
 1/64 ━━━━━━━━━━━━━━━━━━━━ 11s 181ms/stepTraining loss (for one batch) at step 0: 0.0001
Seen so far: 64 samples
66/64 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/steploss: 0.32944345474243164,recall: 0.8198661804199219


2025-02-24 03:43:01.247866: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


val_loss: 0.6875649690628052,val_recall: 1.0
64/64 ━━━━━━━━━━━━━━━━━━━━ 12s 195ms/step - loss: 0.3294 - val_loss: 3.3179e-04


In [7]:
import numpy as np


np.bincount(Y_train)

NameError: name 'Y_train' is not defined